# Submission - Exercises sheet 2
MA-INF 4236 - Advanced Methods for Text Mining, So24<br>
Yara Elwakeel 50135730<br>
Aksa Aksa 50146305<br>
Sadia Naseer 50194450<br>
Ali Ather 50194294<br>
Affan Zafar 50167759<br>
june 19, 2024<br>

## 1)

## 2)

## 3)

## 4) 

## 5) 

## 6) 

In [ ]:
# Install required packages
!pip install -q sentence-transformers datasets

In [3]:
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import numpy as np

# Load the dataset
dataset = load_dataset('mt0rm0/movie_descriptors_small')

# print(dataset['train'])
# Extract movie titles and synopses
movie_titles = [entry['title'] for entry in dataset['train']]
movie_synopses = [entry['overview'] for entry in dataset['train']]

In [4]:
# Load the SentenceBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all movie synopses
synopsis_embeddings = model.encode(movie_synopses, convert_to_tensor=True)

c:\Users\yarae\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def recommend_movies(query, top_k=5):
    # Compute the embedding for the query
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Compute cosine similarities between the query embedding and all synopsis embeddings
    similarities = util.pytorch_cos_sim(query_embedding, synopsis_embeddings)[0]
    
    # Get the indices of the top_k most similar synopses
    top_k_indices = similarities.topk(k=top_k).indices
    
    # Retrieve and return the corresponding movie titles
    recommended_movies = [movie_titles[idx] for idx in top_k_indices]
    return recommended_movies


In [6]:
# Example queries
queries = [
    "What movie is set in ancient Rome (similar to Gladiator)?",
    "Please recommend me some movies that are both romantic and funny.",
    "Which movies are about astronauts and starships?",
    "Looking for a thrilling heist movie.",
    "Can you suggest a movie with a strong female lead?",
    "I want to watch a movie about artificial intelligence.",
    "Recommend me some heartwarming family movies.",
    "What's a good mystery film with lots of twists?",
    "Suggest some films based on true stories.",
    "Any movies with epic fantasy battles?"
]

# Test the system
for query in queries:
    print(f"Query: {query}")
    recommendations = recommend_movies(query)
    for idx, movie in enumerate(recommendations):
        print(f"{idx+1}. {movie}")
    print("\n")


Query: What movie is set in ancient Rome (similar to Gladiator)?
1. Agora
2. Gladiator
3. Cleopatra
4. La Dolce Vita
5. The Man Who Knew Too Much


Query: Please recommend me some movies that are both romantic and funny.
1. About Last Night
2. Midnight in Paris
3. Hysteria
4. Closer
5. Last Night


Query: Which movies are about astronauts and starships?
1. Forbidden Planet
2. Galaxy Quest
3. You Only Live Twice
4. Treasure Planet
5. The Right Stuff


Query: Looking for a thrilling heist movie.
1. The Getaway
2. The Collector
3. Drive
4. Baby Driver
5. The Score


Query: Can you suggest a movie with a strong female lead?
1. Swimming Pool
2. To Die For
3. Thelma & Louise
4. The Young Victoria
5. Under the Skin


Query: I want to watch a movie about artificial intelligence.
1. Pi
2. Ex Machina
3. King Kong
4. Ghost in the Shell 2: Innocence
5. Gattaca


Query: Recommend me some heartwarming family movies.
1. Midnight in Paris
2. This Is Where I Leave You
3. Legends of the Fall
4. Scary Mo